Q1:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

In [ ]:
X_train = np.arange(0,100,0.5) 
y_train = np.sin(X_train)

X_test = np.arange(100,200,0.5) 
y_test = np.sin(X_test)

n_features = 1

train_series = y_train.reshape((len(y_train), n_features))
test_series  = y_test.reshape((len(y_test), n_features))

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

look_back  = 20
train_generator = TimeseriesGenerator(train_series, train_series,
                                      length        = look_back, 
                                      sampling_rate = 1,
                                      stride        = 1,
                                      batch_size    = 10)
test_generator = TimeseriesGenerator(test_series, test_series,
                                      length        = look_back, 
                                      sampling_rate = 1,
                                      stride        = 1,
                                      batch_size    = 10)

In [ ]:
def make_LSTM_model():
    n_neurons  = 2
    model = Sequential()
    model.add(LSTM(n_neurons, input_shape=(look_back, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
model = make_LSTM_model()
model.fit_generator(train_generator,epochs=40, verbose=0)

In [ ]:
test_predictions  = model.predict_generator(test_generator)

In [ ]:
x = np.arange(110,200,0.5)
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(X_train,y_train, lw=2, label='train data')
ax.plot(X_test,y_test, lw=3, c='y', label='test data')
ax.plot(x,test_predictions, lw=3, c='r',linestyle = ':', label='predictions')
ax.legend(loc="lower left")
plt.show()

In [ ]:
def make_RNN_model():
    n_neurons  = 2
    model = Sequential()
    model.add(SimpleRNN(n_neurons, input_shape=(look_back, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
model = make_RNN_model()
model.fit_generator(train_generator,epochs=15, verbose=0)

In [ ]:
test_predictions  = model.predict_generator(test_generator)

In [ ]:
x = np.arange(110,200,0.5)
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(X_train,y_train, lw=2, label='train data')
ax.plot(X_test,y_test, lw=3, c='y', label='test data')
ax.plot(x,test_predictions, lw=3, c='r',linestyle = ':', label='predictions')
ax.legend(loc="lower left")
plt.show()

Q2:

In [ ]:
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

RF Regression

In [ ]:
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
regr.fit(x_train, y_train)

In [ ]:
predictions = regr.predict(x_test)

In [ ]:
predictions

In [ ]:
mean_squared_error(y_true=y_test, y_pred=predictions)

RF Classification

In [ ]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
clf

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
predictions = clf.predict(x_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=predictions))

Q3:

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Covid cases are increasing fast!"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Q4:

In [ ]:
from sklearn.datasets import load_sample_images
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from PIL import Image as im
import os

In [ ]:
data = load_sample_images()
print(data)

In [ ]:
for i in data['images']:
    pyplot.matshow(i)

In [ ]:
data.keys()

In [ ]:
len(data['images'])

In [ ]:
im_1 = data['images'][0]
im_2 = data['images'][1]

In [ ]:
np.shape(im_1)

Image Reshaping for Data Augmentation

In [ ]:
reshaped_im_1 = im_1.reshape(1,427,640,3)
reshaped_im_2 = im_2.reshape(1,427,640,3)

In [ ]:
np.shape(reshaped_im_1)

In [ ]:
reshaped_im_1.dtype

In [ ]:
reshaped_im_1 = reshaped_im_1.astype("float32")
reshaped_im_2 = reshaped_im_2.astype("float32")

Data Augmentation

In [ ]:
def image_genrator(no_of_images, directory, augmentation_type, iterator):
    for i in range(no_of_images):
        batch = iterator.next()
        image_aray = batch[0].astype('uint8')
        image = im.fromarray(image_aray)
        image.save(os.path.join(directory,str(i)+ augmentation_type+".png"))

def image_augment(image=None, augmentation_type=None, directory=None, no_of_images=0):
    if augmentation_type=="FSA":
        datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    
    elif augmentation_type=="Shift":
        shift = 0.3
        datagen = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift)
        
    elif augmentation_type=="Flip":
        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

    elif augmentation_type=="RandomRot":
        datagen = ImageDataGenerator(rotation_range=90)
    
    datagen.fit(image)
    iterator = datagen.flow(image, batch_size=1, shuffle=False)
    image_genrator(no_of_images=no_of_images, directory=directory, augmentation_type=augmentation_type, iterator=iterator)

Generating 0 class Augmented Images

In [ ]:
image_augment(image=reshaped_im_1, augmentation_type="FSA", directory="augmented_dataset/0", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_1, augmentation_type="Shift", directory="augmented_dataset/0", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_1, augmentation_type="Flip", directory="augmented_dataset/0", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_1, augmentation_type="RandomRot", directory="augmented_dataset/0", no_of_images=5)

Generating 1 class Augmented Images

In [ ]:
image_augment(image=reshaped_im_2, augmentation_type="FSA", directory="augmented_dataset/1", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_2, augmentation_type="Shift", directory="augmented_dataset/1", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_2, augmentation_type="Flip", directory="augmented_dataset/1", no_of_images=5)

In [ ]:
image_augment(image=reshaped_im_2, augmentation_type="RandomRot", directory="augmented_dataset/1", no_of_images=5)

CNN Based Image Classification

In [ ]:
def load_files(data_folder):
    files = list()
    labels = list()
    for folder in os.listdir(data_folder):
        for file in os.listdir(os.path.join(data_folder,folder)):
            file_name = os.path.join(data_folder, folder, file)
            files.append(file_name)
            labels.append(folder)
    return files, labels

In [ ]:
file_names, label = load_files("augmented_dataset")

In [ ]:
print(file_names[:5])
print(label[:5])

In [ ]:
import pandas as pd

def data_loader(file_paths, labels):
    data_frame = pd.DataFrame(columns=["images", "label"])
    for label, file in enumerate(file_paths):
        
        img = im.open(file)
        img = np.array(img)
        
        data_frame.loc[len(data_frame.index)] = [img, labels[label]]
    return data_frame

In [ ]:
df = data_loader(file_paths=file_names, labels=label)

Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df["images"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, shuffle=True)

In [ ]:
type(X_train[0])

In [ ]:
X_train[0].shape

In [ ]:
print(y_train.values)
y_train = y_train.values.astype(int)
print(y_train)
y_test = y_test.values.astype(int)

Reshaping for Keras

In [ ]:
X_train = [img for img in X_train]
X_train = np.array(X_train)
print(X_train.shape)

X_test = [img for img in X_test]
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_test.shape)

Model

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu',input_shape=(427, 640, 3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

Training

In [ ]:
epochs=1

history = model.fit(X_train, y_train, epochs=epochs, batch_size=4).history

Testing

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction

In [ ]:
prediction = np.where(prediction > 0.5, 1,0)

In [ ]:
prediction

Performance Evaluation

In [ ]:
from sklearn.metrics import classification_report as cr
report = cr(y_true=y_test, y_pred=prediction)
print(report)

Save Keras Mode

In [ ]:
model.save("./keras_model.model")

Model Deployment using TF-Lite

In [ ]:
import tensorflow
import keras

In [ ]:
model = keras.models.load_model("./keras_model.model")

converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)